In [1]:
import re
import json
import csv
import requests
from bs4 import BeautifulSoup ,SoupStrainer
import pandas as pd 
from io import StringIO

!pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import sys

import unittest

import time


In [31]:
def get_infinity_page_html_data(chromedriver_path="./chromedriver.exe", scroll_number=10, sleep_time=2, url = ""):
    # Create selenium driver
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.get(url)

    for i in range(1,scroll_number):
        print(f"scroll_number: {i}")
        driver.execute_script("window.scrollTo(1,50000)")
        time.sleep(sleep_time)
    html_data = driver.page_source
    driver.close()
    
    soup = BeautifulSoup(html_data, 'html.parser')    
    return soup


In [32]:
soup = get_infinity_page_html_data(url = "https://finance.yahoo.com/quote/AAPL/history?period1=345427200&period2=1651881600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true")


scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_number: 8
scroll_number: 9


In [33]:
print(soup.prettify)


<bound method Tag.prettify of <html class="chrome desktop JsEnabled themelight layoutEnhance(TwoColumnLayout) CollapsibleUh onDemandFocusSupport HideNavrail hasScrolled scrollDown" data-color-theme="light" id="atomic" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>Apple Inc. (AAPL) Stock Historical Prices &amp; Data - Yahoo Finance</title><meta content="AAPL, Apple Inc., AAPL historical prices, Apple Inc. historical prices, historical prices, stocks, quotes, finance" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" property="twitter:dnt"/><meta content="458584288257241" property="fb:app_id"/><meta content="#400090" name="theme-color"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Discover historical prices for AAPL stock on Yahoo Finance. View daily, weekly or monthly form

In [47]:
table = soup.find_all('table')
df_Apple = pd.read_html(str(table))[0]
df_Apple


,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"May 06, 2022",156.01,159.44,154.18,157.28,157.28,116055700
1,"May 06, 2022",0.23 Dividend,0.23 Dividend,0.23 Dividend,0.23 Dividend,0.23 Dividend,0.23 Dividend
2,"May 05, 2022",163.85,164.08,154.95,156.77,156.54,130525300
3,"May 04, 2022",159.67,166.48,159.26,166.02,165.78,108256500
4,"May 03, 2022",158.15,160.71,156.32,159.48,159.25,88966500
...,...,...,...,...,...,...,...
996,"Jun 18, 2018",46.97,47.31,46.80,47.19,45.36,73939600
997,"Jun 15, 2018",47.51,47.54,47.06,47.21,45.39,246876800
998,"Jun 14, 2018",47.89,47.89,47.56,47.70,45.86,86440400
999,"Jun 13, 2018",48.10,48.22,47.61,47.67,45.83,86553600


In [48]:
df_Apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         1001 non-null   object
 1   Open         1001 non-null   object
 2   High         1001 non-null   object
 3   Low          1001 non-null   object
 4   Close*       1001 non-null   object
 5   Adj Close**  1001 non-null   object
 6   Volume       1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [50]:
df_Apple_cpy = df_Apple.copy()
df_Apple_cpy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         1001 non-null   object
 1   Open         1001 non-null   object
 2   High         1001 non-null   object
 3   Low          1001 non-null   object
 4   Close*       1001 non-null   object
 5   Adj Close**  1001 non-null   object
 6   Volume       1001 non-null   object
dtypes: object(7)
memory usage: 54.9+ KB


In [71]:
df_Apple = df_Apple[df_Apple["Open"].str.contains("Dividend") == False]
df_Apple.drop(df_Apple.tail(1).index,inplace=True)
df_Apple.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 984 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         984 non-null    object
 1   Open         984 non-null    object
 2   High         984 non-null    object
 3   Low          984 non-null    object
 4   Close*       984 non-null    object
 5   Adj Close**  984 non-null    object
 6   Volume       984 non-null    object
dtypes: object(7)
memory usage: 61.5+ KB


In [73]:
#df_Apple = 
#df_Apple.head()
df_Apple

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"May 06, 2022",156.01,159.44,154.18,157.28,157.28,116055700
2,"May 05, 2022",163.85,164.08,154.95,156.77,156.54,130525300
3,"May 04, 2022",159.67,166.48,159.26,166.02,165.78,108256500
4,"May 03, 2022",158.15,160.71,156.32,159.48,159.25,88966500
5,"May 02, 2022",156.71,158.23,153.27,157.96,157.73,123055300
...,...,...,...,...,...,...,...
995,"Jun 19, 2018",46.28,46.58,45.86,46.42,44.63,134314000
996,"Jun 18, 2018",46.97,47.31,46.80,47.19,45.36,73939600
997,"Jun 15, 2018",47.51,47.54,47.06,47.21,45.39,246876800
998,"Jun 14, 2018",47.89,47.89,47.56,47.70,45.86,86440400


In [77]:
from pathlib import Path  
filepath = Path('out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_Apple.to_csv(filepath) 

In [24]:
base_url = "https://finance.yahoo.com/quote/"
period_url = "/history?period1=345427200&period2=1651881600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT"]
main_df = 0
for index in indexes:
    soup = get_infinity_page_html_data(url = (base_url + index + period_url))
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    print(df.head())


scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
           Date    Open    High     Low  Close* Adj Close**    Volume
0  May 06, 2022  187.36  195.12  179.90  186.75      186.75  63299300
1  May 05, 2022  198.67  199.25  185.00  188.44      188.44  62633100
2  May 04, 2022  199.23  204.00  187.51  203.34      203.34  64885500
3  May 03, 2022  194.00  198.25  191.33  196.02      196.02  47575100
4  May 02, 2022  185.41  195.74  183.91  195.33      195.33  57204900
scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
           Date           Open           High            Low         Close*  \
0  May 06, 2022         156.01         159.44         154.18         157.28   
1  May 06, 2022  0.23 Dividend  0.23 Dividend  0.23 Dividend  0.23 Dividend   
2  May 05, 2022         163.85         164.08         154.95         156.77   
3  May 04, 2022         159.67         166.48         159.26         166.02   
4  May 03, 2022         158.15         160.71    

WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.54)
Stacktrace:
Backtrace:
	Ordinal0 [0x00267413+2389011]
	Ordinal0 [0x001F9F61+1941345]
	Ordinal0 [0x000EC520+836896]
	Ordinal0 [0x000E0682+788098]
	Ordinal0 [0x000E0EB8+790200]
	Ordinal0 [0x000E2752+796498]
	Ordinal0 [0x000DC0D9+770265]
	Ordinal0 [0x000ED9D0+842192]
	Ordinal0 [0x00143AE2+1194722]
	Ordinal0 [0x00133F66+1130342]
	Ordinal0 [0x0010E546+976198]
	Ordinal0 [0x0010F456+980054]
	GetHandleVerifier [0x00419632+1727522]
	GetHandleVerifier [0x004CBA4D+2457661]
	GetHandleVerifier [0x002FEB81+569713]
	GetHandleVerifier [0x002FDD76+566118]
	Ordinal0 [0x00200B2B+1968939]
	Ordinal0 [0x00205988+1989000]
	Ordinal0 [0x00205A75+1989237]
	Ordinal0 [0x0020ECB1+2026673]
	BaseThreadInitThunk [0x76E7FA29+25]
	RtlGetAppContainerNamedObjectPath [0x778C7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x778C7A4E+238]
	(No symbol) [0x00000000]


In [ ]:
###############################################################################################

#getting Historical data page"
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }

Main_url ='https://finance.yahoo.com/quote/NDAQ?p=NDAQ&.tsrc=fin-srch'
r = requests.get(Main_url)
soup = BeautifulSoup( r.content , 'html.parser')

for name in soup.find_all('ul',attrs = {"class": "List(n) Whs(nw) fin-tab-items W(100%) Lh(1.7) H(44px) Bdbs(s) BdB(4px) Cf Mb(15px) Bdbc($seperatorColor)" }):
    for link in name.find_all('a', text = "Historical Data"): 
        links = link.get('href')
Hiso_URL = "https://finance.yahoo.com" + links 
print(Hiso_URL)



In [ ]:
for name in soup.find_all('div',attrs = {"class": "C($tertiaryColor) Mt(20px) Mb(15px)"}):
    print(lin)
    


In [ ]:
#Getting to Dta

resp = requests.get(Hiso_URL, headers=headers, timeout=5)
soup1 = BeautifulSoup(resp.content , 'html.parser')

for name in soup1.find_all('div',attrs = {"class": "C($tertiaryColor) Mt(20px) Mb(15px)"}):
    lin = name.get('href')
    
print(lin)
  #--------------------------#
link = 'https://query1.finance.yahoo.com/v7/finance/download/NDAQ?period1=1025481600&period2=1651104000&interval=1d&events=history&includeAdjustedClose=true'
            #Need to figure out why link is not there
link = 'https://query1.finance.yahoo.com/v7/finance/download/%5ERUT?period1=558230400&period2=1651104000&interval=1d&events=history&includeAdjustedClose=true'    
#link = 'https://query1.finance.yahoo.com/v7/finance/download/NDAQ{}?'

#response = requests.get(link , headers=headers, timeout=5)

#params = {

#'period1': '%s' %int(past_date.timestamp()),
#'period2': '%s' %int(dt.timestamp()),
#'events': 'history',
#'includeAdjustedClose': 'true'

#}

#response = requests.get(link , params = params ,headers=headers, timeout=5)

response = requests.get(link , headers=headers, timeout=5)

file = StringIO(response.text)
reader =csv.reader(file)
df = pd.read_csv(file)



In [ ]:
df.drop(["Volume","Adj Close"], axis=1, inplace=True)

In [ ]:
#######feature engineering##############

df['Dayly change'] = (df['Close']-df['Open'])/df['Open']*100
df['Dayly change Max'] =  abs((df['High']-df['Low'])/df['Low']*100)

In [ ]:
df

